In [4]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.6 MB 991.0 kB/s eta 0:00:02
   --------- ------------------------------ 0.4/1.6 MB 4.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.1/1.6 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 9.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd
import zipfile
import requests
from rapidfuzz import process

def get_free_city_list():
    """
    Obtiene una lista de ciudades válidas desde una fuente gratuita como GeoNames o SimpleMaps.
    """
    url = "https://simplemaps.com/static/data/world-cities/basic/simplemaps_worldcities_basicv1.75.zip"
    
    # Descargar y extraer el archivo ZIP
    zip_file_path = "simplemaps_worldcities_basicv1.75.zip"
    with requests.get(url) as r:
        with open(zip_file_path, 'wb') as f:
            f.write(r.content)
    
    # Extraer el archivo Excel del ZIP
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extract('worldcities.xlsx')  # Especificar el archivo Excel
    
    # Leer el archivo Excel con pandas
    df_cities = pd.read_excel('worldcities.xlsx', usecols=[0])  # Suponiendo que la primera columna tiene los nombres de ciudades
    return df_cities["city"].dropna().str.title().tolist()

def is_valid_city(city_name, valid_city_list):
    """
    Verifica si el nombre es una ciudad válida comparándolo con una lista de ciudades conocidas.
    """
    result = process.extractOne(city_name, valid_city_list, score_cutoff=85)
    
    # Verificar si se encontró un match
    if result:
        match, score, _ = result  # extraemos solo los dos primeros valores (match y score)
        return match is not None
    return False

def standardize_city_names(df, column_name, valid_city_list):
    """
    Estandariza los nombres de ciudades en una columna específica de un DataFrame.
    - Convierte a formato título (capitalizando cada palabra).
    - Elimina espacios innecesarios.
    - Corrige nombres similares usando coincidencia difusa.
    - Filtra registros que no sean ciudades válidas.
    - Conserva la columna con el nombre original.
    """
    df["Original_" + column_name] = df[column_name]  # Guardar la columna original
    df[column_name] = df[column_name].str.strip().str.title()
    
    # Filtrar las ciudades válidas
    df = df[df[column_name].apply(lambda city: is_valid_city(city, valid_city_list))]
    
    unique_cities = df[column_name].unique()
    standardized_names = {}
    
    for city in unique_cities:
        result = process.extractOne(city, standardized_names.keys(), score_cutoff=85)
        if result:
            match, _, _ = result  # Extraer solo el match
            standardized_names[city] = standardized_names[match]
        else:
            standardized_names[city] = city
    
    df[column_name] = df[column_name].map(standardized_names)
    return df

# Cargar el archivo Excel
df = pd.read_excel("ciudades.xlsx")

# Obtener lista de ciudades válidas de una fuente gratuita
valid_city_list = get_free_city_list()

# Aplicar la estandarización y filtrado
df = standardize_city_names(df, "City", valid_city_list)

# Guardar el archivo con los nombres estandarizados
df.to_excel("ciudades_estandarizadas.xlsx", index=False)

print("Archivo procesado y guardado como 'ciudades_estandarizadas.xlsx'")

C:\Users\User\AppData\Local\Temp\ipykernel_6520\719342158.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].map(standardized_names)


Archivo procesado y guardado como 'ciudades_estandarizadas.xlsx'


In [12]:
# Cargar el archivo Excel
df = pd.read_excel("ciudades2.xlsx")

# Obtener lista de ciudades válidas de una fuente gratuita
valid_city_list = get_free_city_list()

# Aplicar la estandarización y filtrado
df = standardize_city_names(df, "City", valid_city_list)

# Guardar el archivo con los nombres estandarizados
df.to_excel("ciudades_estandarizadas2.xlsx", index=False)

print("Archivo procesado y guardado como 'ciudades_estandarizadas2.xlsx'")

Archivo procesado y guardado como 'ciudades_estandarizadas2.xlsx'


C:\Users\User\AppData\Local\Temp\ipykernel_6520\719342158.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].map(standardized_names)
